# Notes on Python
## Examples for reference, tips, Best Practices

Based on the Course: Core Python (Introspection and Numeric Types, Times and Dates) at PluralSight

Author: Gonçalo Felício  
Date: 04/2022  
Provided by: ISIWAY

Something like a pocketbook to come to for quick references, examples, and tips of best practices, compiled with my own preferences  
Loosely divided by subject, and with some degree, by the respective modules

## Introspection

#### Types
Introspection regards inspecting objects to better understand how they work, and how to use them, in specific their atributes

`type()` is the built in function that returns the type of its argument  
Types of objects are usually expressed in terms of `class`  

`issubclass()` reports wether one type is a class of another  
`isinstance()` reports wether an object is an instance of a type  
When type check is necessary, it's better to use these functions rather than equality checks

`dir()` lists all the attributes of an object, this includes methods, if querying a class  
Usually, a type will have attributes that are not immediately clear why this type would include them. For example `int` includes attributes that allow us to use objects of this type as rational or complex numbers  
`getattr()` allows us to access attributes directly by string name, if an attribute of this name does not exist a `AttributeError` is raised  
`callable()` determines if an object can be called like a function  
The `__name__` and `__class__` contain the name of the class object and the type information respectively
`hasattr()` checks wether an object contains an atribute with the argument string name

Tip: Even though introspection is a powerfull tool for development, it's better to use a EAFP style of programming, as it is often faster and cleaner

This example uses the attributes `numerator` and `denominator` of int and fraction objects to return mixed numerals  
Notice how we could check if the argument has the attributes necessary, but instead, it's cleaner and faster to simply try the implementation and catch the `AttributeError` if the argument does lack the attributes  
At this point we change the unclear `AttributeError` into the more common `TypeError` and return a helpful message using the Exception syntax `except Error as e: raise ChainedError ... from e` for chained exceptions

In [2]:
from fractions import Fraction


def mixed_numeral(vulgar):
    try:
        integer = vulgar.numerator // vulgar.denominator
        fraction = Fraction(vulgar.numerator - (integer * vulgar.denominator), vulgar.denominator)
        
        return integer, fraction
    except AttributeError as e:
        raise TypeError(f"{vulgar} is not a rational number") from e

In [4]:
mixed_numeral(Fraction('11/10'))

(1, Fraction(1, 10))

In [6]:
mixed_numeral(15)

(15, Fraction(0, 1))

In [5]:
mixed_numeral(1.1)

TypeError: 1.1 is not a rational number

#### Scopes
`globals()` is a dictionay mapping names to objects in the global namespace, so, in other words, the dictionary returned is the actual global namespace  
`locals()` is the same but for the local scope  

Tip: it's generally better to use f-strings to access the local or global namespace variables directly


#### Inspect Module
The inspect module provides many useful tools for advanced introspection  
`inspect.ismodule()` checks if an object is a module  
`inspect.getmembers()` returns the members of an object and can accept arguments for filtering the results  
`inspect.signature()` returns information on a function's signature or metadata, if this information is not accessible it raises a `ValueError`  
`inspect.getdoc()` returns nicely formatted docstrings of an object

Importing a module binds the module's namespace into the current namespace, this means we can actually import modules that are imported in other modules directly from the latter

In [ ]:
import inspect

inspect.getmembers(batch, inspect.isclass) # returns all the classes inside the module batch

In [ ]:
inspect.getmembers(batch.Batch, inspect.isfunction) # returns all the functions inside the class Batch of module batch

In [ ]:
init_sig = inspect.signature(batch.Batch.__init__) # returns the signature of the funtion __init__ of class Batch

In [ ]:
str(init_sig) # returns a nice consice view of the information

A common guideline for docstrings is that the first line is a short summary  
`set` objects can be used to work with relationships between groups of objects as in handling the names of the attributes of an object wether they are attributes or methods  
When deciding between using the `map` function or a comprehension, choose the one that increases clarity and readability  
Format specifiers in `.format()` can be nested inside the string like so `'widths: {{:{w}}}'` and the results is `'widths: {:3} {:6} {:9}...'`



## Numeric Types, Dates and Time
Sometimes the usual numeric types like `int` and `float` are not suitable for an implementation and can lead to errors. At this point special numeric types should be adopted

`float` uses 64 bits to store its value, 1 bit for the sign, 11 bits for the exponent and 52 bits for the mantissa
`float` can't exactly represent every floating-point number, as seen in simple example below. These imprecisions can accumulate and lead to significant errors in implementations



In [8]:
0.8-0.7 # notice the imprecision at the last value

0.10000000000000009

### Decimal
The Decimal type can be used to exactly represent decimal values, it uses a base-10 floating-point number representation  
Decimal can be safely constructed with strings and integers  
Constructing Decimal from float can lead to loss of data  
Decimal preserves precision between computations and the precision can be specified in the context of Decimal, it also supports 'infinity' and 'not-a-number'  
Functions in `math` don't work with Decimal

In [13]:
from decimal import Decimal, getcontext
Decimal('0.8')-Decimal('0.7')

Decimal('0.1')

In [14]:
getcontext().prec = 6
D = Decimal('1.2345678')
D

Decimal('1.2345678')

In [17]:
D + Decimal(2) # precision is maintained at the specified level

Decimal('3.23457')

### Fraction
Fraction allows for precise representation of rational numbers 
It can be constructed in the following ways  
Take care when using floats as it can carry over it's inexact nature  
Supports standard arithmetic, and the `ceil()` and `floor()` functions, but not `sqrt()` as it could lead to irrational numbers

In [21]:
from fractions import Fraction

print(Fraction(1,2))
print(Fraction(0.5))
print(Fraction(0.1))
print(Fraction(Decimal('0.1')))
print(Fraction('0.6'), Fraction('22/4'))

1/2
1/2
3602879701896397/36028797018963968
1/10
3/5 11/2


### Complex
complex is the way to model numbers with imaginary parts  
It can be constructed from other numeric types and stores the components as floats  
Functions from `math` module do not work but there are alternatives in the `cmath` module that substitue

In [23]:
type(2j)

complex

In [27]:
c = 3+5j
print(c)
print(c.real)
print(c.conjugate())

(3+5j)
3.0
(3-5j)


In [28]:
import cmath
cmath.sqrt(-1)

1j

`round()` does not work for complex numbers and can give suprising results for floats

In [32]:
print(round(1.5))
print(round(2.5))
print(round(2.675, 2))
# The way it works is that it rounds to the even number as seen by 1.5 and 2.5
# Still it may work differently as we were expecting 2.68 and got 2.67, meaning it was closer to 2.67 than to 2.68

2
2
2.67


Pyhton has the literal forms for *binary* *octal* and *hexadecimal* using `bin()`, `oct()`, `hex()`  
`int()` accepts the optional base argument to use when interpreting the given string, taking any value from 2 to 36. The default is to use the string prefix, if given or decimal

In [42]:
int('100000', base=36)

60466176

### Datas and Times
`datetime` module provides support to working with dates and times  
`datetime.date` represents calendar dates  
`datetime.time` represents time of day  
`datetime.datetime` represents both  
`datetime.timedelta` represents durations between 2 given times  
`datetime.tzinfo` provides an abstract class to work with timezones  
We can add a timedelta to a datetime to get the new datetime, but we cannot perform basic arithmetics on datetimes   
Tip: Python does not include much support for handling timezones, given its volatility, search for 3rd-party packages for this like `pytz` or `dateutil`

In [54]:
import datetime
td = datetime.timedelta(weeks=3, days=3, hours=3, seconds=3, microseconds=3)
td # curiously only representes days, seconds and microseconds by default

datetime.timedelta(days=24, seconds=10803, microseconds=3)

In [47]:
str(td) # a nicely formatted representation of timedelta

'24 days, 3:00:03.000003'

In [53]:
str(datetime.date.today())

'2022-04-12'

#### Computational Geometry
Computational geometry is an area where the inexactness of floats can lead to major implementation errors

This function is rather slick, it takes advatage of the results from boolean aritmetics, as True == 0 and False == -1

In [68]:
print(
    (True - False),
    (False - True),
    (True - True),
    (False - False), sep='\n')

1
-1
0
0


In [55]:
# Returns the sing of a number
def sign(x):
    return  (x > 0) - (x < 0)

In [70]:
print(sign(2), sign(-3), sign(0), sep='\n')

1
-1
0
